In [14]:
from joblib import load

cv = load('../model/cv.pkl')
model = load('../model/nb.pkl')
bc = load('../model/bc.pkl')

In [15]:
# set file path
filepath = '../data/'
trainfile = 'test.csv'

In [16]:
# read train.csv
import pandas as pd
df = pd.read_csv(filepath+trainfile)
df = df.sample(df.shape[0])

In [17]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

def preprocess(docs):
    def format_text(text):
        text=re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', '', text)
        text=re.sub('RT', '', text)
        text=re.sub(r'[!-/?:@]', '', text)#半角記号,数字,英字
        text=re.sub(r'[︰-＠]', '', text)#全角記号
        text=re.sub('\n', ' ', text)#改行文字
        return text
 
    stopwords_set = set(stopwords.words('english'))
    wordnet_lemmatizer = WordNetLemmatizer()
    
    docs = [format_text(text) for text in docs]
    docs = [[word for word in word_tokenize(text) if word not in stopwords_set] for text in docs]
    docs = [' '.join([wordnet_lemmatizer.lemmatize(word.lower()) for word in doc]) for doc in docs]
    docs = bc.fit_transform(docs)
    
    return cv.transform(docs)

In [18]:
X_test = preprocess(df.text.values.tolist())
pred = model.predict(X_test)

100%|██████████| 3263/3263 [00:00<00:00, 13900.56it/s]


```
id,target
0,0
2,0
3,1
9,0
11,0
```

In [19]:
df = pd.DataFrame({'id': df.id.values, 'target': pred})
df.target.value_counts()

0    2297
1     966
Name: target, dtype: int64

In [20]:
df.to_csv('../output/submit.csv', index=None)